# Retrieving and Aggregating NWM Data with a Feature ID

**Note:** 
We have altered this code so that it only stores daily inflow and the corresponding day in the csv file. The maximum timeframe is used (1980-01-01 to 2023-01-31). 
- Matt Burgos mb2557, Rachel Pyeon rp653


.

.

**Authors:** 

<ul style="line-height:1.5;">
<li>Ayman Nassar <a href="mailto:ayman.nassar@usu.edu">(ayman.nassar@usu.edu)</a></li>
<li>Pabitra Dash <a href="mailto:pabitra.dash@usu.edu">(pabitra.dash@usu.edu)</a></li>
<li>Homa Salehabadi <a href="mailto:homa.salehabadi@usu.edu">(homa.salehabadi@usu.edu)</a></li>
<li>David Tarboton <a href="mailto:david.tarboton@usu.edu">(david.tarboton@usu.edu)</a></li>
</ul>

**Last Updated:** 10/07/2024

**Purpose:**

This notebook provides code examples for retrieving NOAA National Water Model (NWM) Retrospective data from Amazon Web Services (AWS). It makes it easier for researchers to access data with a specific feature ID. It also allows for data aggregation at a different time scale than what NOAA originally provided.

**Audience:**

Researchers who are familiar with Jupyter Notebooks, basic Python and basic hydrologic data analysis.

**Description:**

This notebook takes a feature ID specifying the location, start and end dates for the desired period, a variable name, and a preferred time aggregation interval as inputs. It then retrieves data from AWS, aggregates it over the specified time interval, displays the data as a plot, and saves it as a CSV file.

**Data Description:**

This notebook uses data developed and published by NOAA on Amazon Web Services (AWS) as described in detail in this registry of open data entry <https://registry.opendata.aws/nwm-archive/>. The NOAA National Water Model Retrospective dataset contains input and output from multi-decade CONUS retrospective simulations. These simulations used meteorological input fields from meteorological retrospective datasets. The output frequency and fields available in this historical NWM dataset differ from those contained in the real-time operational NWM forecast model. Additionally, note that no streamflow or other data assimilation is performed within any of the NWM retrospective simulations. This notebook uses the Zarr format version of this data. Zarr is a format for storage of chunked, compressed, N-dimensional arrays, designed to support storage using distributed systems such as cloud object stores (<https://zarr.dev/>).

**Software Requirements:**

This notebook has been tested on the CIROH Jupyterhub, CyberGIS Jupyter for Water and CUAHSI JupyterHub deployments. It relies on the general-purpose Jupyter computing environment with the following specific Python libraries: 

>  numpy: 1.26.4     
   geopandas: 0.14.3  
   pandas: 2.2.1  
   matplotlib: 3.8.3   
   shapely: 2.0.3
 
It also uses code from nwm_utils.py that accompanies this notebook.

### 1. Import Python Libraries Needed to Run this Jupyter Notebook

In [32]:
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box
from mpl_toolkits.axes_grid1 import make_axes_locatable
from nwm_utils import get_conus_bucket_url, load_dataset, reproject_coordinates, get_fid, get_aggregation_code

### 2. Set Inputs

Use the cells in this section of the notebook to set the input values that specify the data to retrieve. Feature ID (fid) for stream reaches may be identified from the NWM map <https://water.noaa.gov/map> stream reach ID. In the map, open the National Water Model layer and enable Stream Reach to zoom in on a reach of interest to obtain its ID. The example below fid = 644150 is for a reach of the Logan River.

In [33]:
# Start date - in Year-Month-Day format, the earliest start date can be '1979-02-01'
start_datetime = '1980-01-01'
# End date - in Year-Month-Day format, the latest end date can be '2023-01-31'
end_datetime = '2023-01-31'

# Feature ID - this is the feature ID used in the NWM
# Note: If you need to retrieve the feature ID (fid) using a USGS Gage ID, you can use the function 'fid = get_fid(usgs_gageid)' which is designed to convert a USGS Gage ID to its corresponding feature ID (fid). 
# For example, 'fid = get_fid("10109000")' will return the feature ID as '664424'
#fid = "664150"
fid = get_fid("01350000")

**Users may select one of the following datasets to retrieve data:**
- Streamflow Output at All Channel Reaches
- Ground water output
- Lake Output

For a full list of output variables, refer to: <https://ral.ucar.edu/sites/default/files/public/WRFHydroV5_OutputVariableMatrix_V5.pdf>

In [34]:
# User-defined NWM output dataset; see above for a list of valid dataset names.
nwm_out_ds = 'Streamflow Output at All Channel Reaches' 

# User-defined variable name; This variable is part of the National Water Model (NWM) output variable matrix. 
# For a full list of NWM output variables, refer to the following website: https://ral.ucar.edu/sites/default/files/public/WRFHydroV5_OutputVariableMatrix_V5.pdf
variable_name = 'streamflow' 

# User-defined aggregation interval - valid values are 'hour','day','month','year'
agg_interval='year'

## 3. Virtually Load the Data Array
This block of code maps the input variable and aggregation interval onto the variable encoding used in the Zarr bucket storage.  It then loads the virtual xarray dataset for the variable of interest. 

In [35]:
# Set dictionary to map output name onto associated code used in the AWS bucket storage
nwm_outputs = {
    'Streamflow Output at All Channel Reaches':'chrtout', 
    'Ground water Output':'gwout', 
    'Lake Output':'lakeout',
}

# Get the code of the NWM output dataset from the dictionary (nwm_outputs)
variable_code = nwm_outputs[nwm_out_ds]

# Get the S3 bucket URL for the data path
url = get_conus_bucket_url(variable_code)

# Load the dataset
ds = load_dataset(url)
print(ds)

<xarray.Dataset>
Dimensions:         (feature_id: 2776734, time: 385704)
Coordinates:
    elevation       (feature_id) float32 dask.array<chunksize=(2776734,), meta=np.ndarray>
  * feature_id      (feature_id) int64 101 179 181 ... 1180001803 1180001804
    gage_id         (feature_id) |S15 dask.array<chunksize=(2776734,), meta=np.ndarray>
    latitude        (feature_id) float32 dask.array<chunksize=(2776734,), meta=np.ndarray>
    longitude       (feature_id) float32 dask.array<chunksize=(2776734,), meta=np.ndarray>
    order           (feature_id) int32 dask.array<chunksize=(2776734,), meta=np.ndarray>
  * time            (time) datetime64[ns] 1979-02-01T01:00:00 ... 2023-02-01
Data variables:
    crs             |S1 ...
    qBtmVertRunoff  (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    qBucket         (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    qSfcLatRunoff   (time, feature_id) float64 dask.array<chunksize=

### 4. Subset and Aggregate Data

Data for the specified variable is retrieved and aggregated to the time interval specified, using sum aggregation Xarray function.  The time step of the input NWM data is hourly.

The results are saved in a data frame ds_subset.df which holds as columns information about the feature of interest and the values for the variable of interest. 

In [36]:
# Subset the data for the specified period
ds_subset = ds.sel(feature_id=np.float64(fid))[variable_name].sel(time=slice(start_datetime, end_datetime))

# Retrieve the variable code from the aggregation dictionary
agg_code = get_aggregation_code(agg_interval)

# Aggregate the data to the specified time interval
ds_subset_df = ds_subset.resample(time=agg_code).sum().to_dataframe()
ds_subset_df

,elevation,feature_id,gage_id,latitude,longitude,order,streamflow
time,,,,,,,
1980-12-31,344.73999,3247466,b' 01350000',42.322849,-74.436333,3,72612.298377
1981-12-31,344.73999,3247466,b' 01350000',42.322849,-74.436333,3,88573.578020
1982-12-31,344.73999,3247466,b' 01350000',42.322849,-74.436333,3,82764.248150
1983-12-31,344.73999,3247466,b' 01350000',42.322849,-74.436333,3,126050.177183
1984-12-31,344.73999,3247466,b' 01350000',42.322849,-74.436333,3,115340.917422
1985-12-31,344.73999,3247466,b' 01350000',42.322849,-74.436333,3,75620.348310
1986-12-31,344.73999,3247466,b' 01350000',42.322849,-74.436333,3,111855.337500
1987-12-31,344.73999,3247466,b' 01350000',42.322849,-74.436333,3,158938.716447
1988-12-31,344.73999,3247466,b' 01350000',42.322849,-74.436333,3,77408.178270


### 5. Plot Data

In [37]:
import pandas as pd
import matplotlib.pyplot as plt

# Label and title
units = ds_subset.attrs.get('units', 'Unknown Units')
long_name = ds_subset.attrs.get('long_name', 'Unknown Variable')

# --- Select one value per day (first record of each day) ---
ds_daily = ds_subset.resample(time='1D').first()

# --- Save x (time) and y (data) to CSV ---
df = ds_daily.to_dataframe().reset_index()

# Identify time and variable columns
time_col = 'time' if 'time' in df.columns else df.columns[0]
value_col = ds_daily.name if ds_daily.name in df.columns else df.columns[-1]

# Create clean DataFrame for export
df_out = df[[time_col, value_col]].copy()
df_out.columns = ['Date', f'{long_name} ({units})']

# Save to CSV
csv_filename = "Schoharie_Creek_at_Prattsville.csv"
df_out.to_csv(csv_filename, index=False)
print(f"Daily (first value) data saved to {csv_filename}")

"""
# --- Plot the daily data ---
plt.figure(figsize=(15, 6))
ds_daily.plot(label=f'{long_name} ({units})')

# Add plot details
plt.title(f'{long_name} Daily Time Series (First Value per Day)')
plt.xlabel('Date')
plt.ylabel(f'{long_name} ({units})')
plt.grid(True)
plt.legend()
plt.tight_layout()

# Display the plot
plt.show()
"""


Daily (first value) data saved to Schoharie_Creek_at_Prattsville.csv


"\n# --- Plot the daily data ---\nplt.figure(figsize=(15, 6))\nds_daily.plot(label=f'{long_name} ({units})')\n\n# Add plot details\nplt.title(f'{long_name} Daily Time Series (First Value per Day)')\nplt.xlabel('Date')\nplt.ylabel(f'{long_name} ({units})')\nplt.grid(True)\nplt.legend()\nplt.tight_layout()\n\n# Display the plot\nplt.show()\n"

### 6. Save Data as a CSV File

In [38]:

# Specify the file path where you want to save the CSV file
#file_path = "new2.csv"

# Save the DataFrame to a CSV file
#ds_subset_df.to_csv(file_path, index=False)  